<a href="https://colab.research.google.com/github/joyful128/Python-Portfolio/blob/main/PPA_Price_Ext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install webdriver_manager
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [ ]:
def extract_market_data(target_date='2023-12-27', target_contract='FTB PPA 24/28'):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=chrome_options)

    try:
        url = f"https://www.omip.pt/en/dados-mercado?date={target_date}&product=EL&zone=ES&instrument=FTB&maturity=PPA"
        driver.get(url)
        print(f"Page loaded successfully for date: {target_date}")
        time.sleep(5)

        rows = driver.find_elements(By.CSS_SELECTOR, "tr.odd, tr.even")
        all_data = []

        print(f"\nLooking for contract: {target_contract}")

        for row in rows:
            try:
                contract_element = row.find_element(By.TAG_NAME, "strong")
                if contract_element:
                    contract_name = contract_element.text
                    if contract_name == target_contract:
                        print(f"Processing contract: {contract_name}")

                        chart_button = row.find_element(By.CSS_SELECTOR, "a.chart-down")
                        chart_url = chart_button.get_attribute("href")

                        # Modify URL for full history
                        full_history_url = chart_url.replace('/0/0', '/0/1')
                        print(f"Full history URL: {full_history_url}")

                        # Get chart data with full history
                        driver.execute_script(f"window.open('{full_history_url}', '_blank');")
                        time.sleep(3)
                        driver.switch_to.window(driver.window_handles[-1])

                        # Extract the JSON data
                        chart_div = driver.find_element(By.CLASS_NAME, "charts-highchart")
                        chart_data = chart_div.get_attribute("data-chart")

                        # Parse the JSON data
                        import json
                        data = json.loads(chart_data)

                        # Extract price series
                        price_series = next(series for series in data['series'] if series['name'] == 'Price (€)')

                        # Convert data points to DataFrame format
                        for point in price_series['data']:
                            timestamp = point[0] / 1000
                            price = point[1]
                            date = pd.to_datetime(timestamp, unit='s')

                            all_data.append({
                                'Date': date,
                                'Price': price
                            })

                        # Close tab and switch back
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                        break

            except Exception as e:
                print(f"Error processing row: {e}")
                continue

        # Create DataFrame
        df = pd.DataFrame(all_data)
        df = df.sort_values('Date')

        print(f"\nDate range for {target_contract}:")
        print("Earliest date:", df['Date'].min())
        print("Latest date:", df['Date'].max())

        return df

    finally:
        driver.quit()

# Run the extraction for a single contract
target_date = '2023-12-27'
target_contract = 'FTB PPA 24/28'  # Just one contract
df = extract_market_data(target_date, target_contract)

# Display the results
print("\nExtracted Data:")
print(df)

# Save to CSV
filename = f'ppa_history_{target_contract}_{target_date}.csv'.replace('/', '_')
df.to_csv(filename, index=False)
from google.colab import files
files.download(filename)

Page loaded successfully for date: 2023-12-27

Looking for contract: FTB PPA 24/28
Error processing row: Message: no such element: Unable to locate element: {"method":"tag name","selector":"strong"}
  (Session info: chrome=132.0.6834.159); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5a537c4fa53a <unknown>
#1 0x5a537bff5f00 <unknown>
#2 0x5a537c045c0c <unknown>
#3 0x5a537c045e31 <unknown>
#4 0x5a537c039866 <unknown>
#5 0x5a537c06a5cd <unknown>
#6 0x5a537c039758 <unknown>
#7 0x5a537c06a76e <unknown>
#8 0x5a537c088f84 <unknown>
#9 0x5a537c06a343 <unknown>
#10 0x5a537c03778a <unknown>
#11 0x5a537c0389de <unknown>
#12 0x5a537c4c42cb <unknown>
#13 0x5a537c4c8242 <unknown>
#14 0x5a537c4b17ac <unknown>
#15 0x5a537c4c8df7 <unknown>
#16 0x5a537c495b2f <unknown>
#17 0x5a537c4e91a8 <unknown>
#18 0x5a537c4e9370 <unknown>
#19 0x5a537c4f93b6 <unknown>
#20 0x7dd3262e0ac3 <unknown>

Err

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>